# SCRAPER TEAMS (FIFAINDEX.COM)

#### Imports

In [ ]:
from bs4 import BeautifulSoup 
import requests
import re
import time
from random import uniform
from tqdm.notebook import tqdm_notebook
import json
from random import choice
from urllib3.util import Retry
from requests.adapters import HTTPAdapter
import urllib3.exceptions
import os
import pandas as pd

## Functions

#### Make Requests

In [ ]:
def make_request(pageURL):
    global errors
    headers_list = [
          {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
           "Accept-Language": "en-US,en;q=0.5",
           "Referer": "https://www.google.com/",
           "DNT": "1",
           "Connection": "keep-alive",
           "Upgrade-Insecure-Requests": "1"
           },
          {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
               "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
               "Accept-Language": "en-US,en;q=0.5",
               "Accept-Encoding": "gzip, deflate, br",
               "Referer": "https://www.google.com/",
               "DNT": "1",
               "Connection": "keep-alive",
               "Upgrade-Insecure-Requests": "1"
           },
          {"Connection": "keep-alive",
               "DNT": "1",
               "Upgrade-Insecure-Requests": "1",
               "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
               "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
               "Sec-Fetch-Site": "none",
               "Sec-Fetch-Mode": "navigate",
               "Sec-Fetch-Dest": "document",
               "Referer": "https://www.google.com/",
               "Accept-Encoding": "gzip, deflate, br",
               "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
           },
          {"Connection": "keep-alive",
               "Upgrade-Insecure-Requests": "1",
               "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
               "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
               "Sec-Fetch-Site": "same-origin",
               "Sec-Fetch-Mode": "navigate",
               "Sec-Fetch-User": "?1",
               "Sec-Fetch-Dest": "document",
               "Referer": "https://www.google.com/",
               "Accept-Encoding": "gzip, deflate, br",
               "Accept-Language": "en-US,en;q=0.9"
           }]
    time.sleep(uniform(0, 2))
    req = requests.Session()
    headers = choice(headers_list)
    retry = Retry(total=8, connect=8, backoff_factor=2)
    req.mount('https://', HTTPAdapter(max_retries=retry))
     
     
    try:
        html = req.get(pageURL, headers=headers, timeout=12)
    except requests.packages.urllib3.exceptions.MaxRetryError:
        errors.append('requests.packages.urllib3.exceptions.MaxRetryError')
        html = requests.get('http://google.com/f')
    except urllib3.exceptions.MaxRetryError:
        errors.append('urllib3.exceptions.MaxRetryError')
        html = requests.get('http://google.com/f')
    except requests.ConnectionError:
        errors.append('requests.ConnectionError')
        html = requests.get('http://google.com/f')
    except urllib3.exceptions.ConnectionError:
        errors.append('urllib3.exceptions.ConnectionError')
        html = requests.get('http://google.com/f')
    except requests.exceptions.ConnectionError:
        errors.append('requests.exceptions.ConnectionError')
        html = requests.get('http://google.com/f')
    except Exception as ex:
        errors.append(str(ex))
        html = requests.get('http://google.com/f')
          
     
    return html

#### Scrape Players' Attributes

In [ ]:
def scrape_team(website_page):
    # Request
    team_html = make_request(pageURL = website_page)
    soup = BeautifulSoup(team_html.text, 'lxml')
    # Team information 
    dd = {}
    dd['TeamName'] = (re.sub(' FIFA.+', '', soup.h1.text).strip()) # team name
    # Observation Date
    for date in soup.select('.dropdown:nth-child(3) .dropdown-toggle'):
        dd['ObservationDate'] = date.text.strip()
    # Rival team
    if soup.find('a', class_='link-team').text is not None:
        dd['RivalTeam'] = soup.find('a', class_='link-team').text
    # Team attributes
    for i in range(2,5): 
        for info in soup.select('.list-group-item:nth-child(' + str(i) + ')'):
            match = re.match("(\D+)(\d{1,2})", info.text, re.I)
            dd[match.group(1)] = match.group(2)
    # Transfer budget
    tranfer_budget = soup.find('span',class_ = 'data-currency-euro').text 
    dd['TransferBudget'] = int(re.sub(r'[.€]', '', tranfer_budget))
    # Team Attributes + Players Roles 
    list1, list2 = [], []
    for value in soup.select('.card-body p'):
        match = re.sub('\d', '', value.text)
        list1.append(match.strip())
    for i in range(1,3): # exception in 2010-2018 period
        if list1[i] == 'Passing':
            list1[i] = 'BuildupPassing'
    if list1[1] == 'Width': # exception in 2019- period
        list1[1] = 'DefensiveWidth'
    for vv in soup.select('.card-body .float-right'):
        list2.append(vv.text)
    for i in range(len(list1)):
        if re.search(list2[i], list1[i]):
            dd[re.sub(list2[i], '', list1[i]).strip()] = list2[i]
        else:
            dd[list1[i]] = list2[i]
    # Team Players + Loaned Players
    players, loaned_players = [], []
    for player in soup.select('td:nth-child(6) .link-player'):
        players.append(player.text)
    dd['TeamRoster'] = players
    for loanedplayer in soup.select('td:nth-child(4) .link-player'):
        loaned_players.append(loanedplayer.text)
    dd['LoanedPlayers'] = loaned_players

    return dd

# Final Results

In [ ]:
errors = []
all_pages = set()
player_links = set()

In [ ]:
j_file = open('chunk10.json')
all_teams_pages = json.load(j_file)
j_file.close()

In [ ]:
len(all_teams_pages)

In [ ]:
# get a list of dicts with all players from fifa10 to fifa22
final_list = []
count = 0
for team_page in tqdm_notebook(all_teams_pages, total=len(all_teams_pages)): 
    try:
        final_list.append(scrape_team('https://www.fifaindex.com' + team_page))
    except ValueError:
        count += 1
        final_list.append({'ERROR': team_page})    

In [ ]:
len(all_teams_pages) == len(final_list)

In [ ]:
df = pd.DataFrame(final_list)
df.to_csv("Teamdata_chunk10.csv", encoding='utf-8', index=False)